In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.python.keras import backend as K
import tensorflow as tf
import collections
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from pandas import ExcelWriter
import pandas.io.formats.excel
from funciones import ErrorAuno,ErrorAcero

from tensorflow.keras.layers import (Activation, AveragePooling2D, BatchNormalization, Cropping2D,
                                     Concatenate, Conv1D, Conv2D, Dense, DepthwiseConv2D, Dropout,
                                     Embedding, Flatten, GlobalAveragePooling2D, Lambda, MaxPool2D,
                                     MaxPooling1D, ReLU, Reshape, ZeroPadding2D)

In [7]:
##### Funciones para trabajar cada caso según el tipo de error si el mismo está dado por la escritura de un 0 cuando
##### debía ser un 1(ErrorAcero) o viceversa. Se ha desarrollado un algoritmo que permite colocar luego del primer bit con
####error encontrado en el resto de las posiciones a 0 o a unos : Se analizan los valores comenzando por el BMS(Bit más
#signif)comparando el valor con el error y el valor que se debe escribir correcto donde haya un cambio de valor se analiza
#### si el cambio es de un 0 x 1 se coloca el resto de los bits a 0, y luego se inyectan los demás errores y si el cambio
#### es de un 1 x 0 se colocan los bits siguientes a 1 y se mantienen los errores restantes haciendo un Bitwise con los
#valores  generados por errores estaticos a 1 o a 0

In [3]:
# A las funciones llegan los valores que deben de gestionarse según cada caso así como las máscara correspondiente que 
#define la posición del error , en el primer caso para poder llevar el resto de los bits a 1 se le resta 1 a la máscara y 
#se hace el bitwise (btw) con ese nuevo valor luego para colocar el resto de los errores se realiza el btw_and(error_0)
#btw_or(error_1)
#def ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change):
#    print('values_to_change dentro de funcion erro_',values_to_change)
#    print('static_0_Error_change',val_afec_0_Error_to_change)
#    print('static_1_Error_change',val_afec_1_Error_to_change)
#    print('values_to_change dentro de funcion erro_0',values_to_change)
#    
#    
#    #tensor_final=tf.where(tf.math.not_equal(vb_and_mask,vm_and_mask) )
#    n_mask=mask-1
#    print('mask', n_mask)
#    n_v_b_m=tf.bitwise.bitwise_or(values_to_change,n_mask)
#    print('n_v_b_m',n_v_b_m)
#    tensor=tf.bitwise.bitwise_and(n_v_b_m,val_afec_0_Error_to_change)
#    print('n_v_b_1', tensor)
#    tensor_round=tf.bitwise.bitwise_or(tensor,val_afec_1_Error_to_change)
#    print( 'tensor_round',tensor_round)
#    #tensor_round  = tf.where(tf.greater_equal(tensor_round,shift), shift-tensor_round , tensor_round )
#    
#    return tensor_round
#    
#    
## En este caso sele resta 1 a la máscara pero debe hacerse un not_lógico a la misma para para cuando se haga el btw_and
##colocar el retso de los bit luego de la posición con error a 0, lo demás es como lo anterior.
#def ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change):
#    print('values_to_change dentro de funcion erro_1',values_to_change)
#    print('static_0_Error_change',val_afec_0_Error_to_change)
#    print('static_1_Error_change',val_afec_1_Error_to_change)
#    
#    #tensor_final=tf.where(tf.math.not_equal(vb_and_mask,vm_and_mask) )
#    n_mask=mask-1
#    print('mask', mask)
#    print('n_mask',n_mask)
#    not_mask=np.invert(np.array([n_mask], dtype=np.uint16))
#    print('not_mask',not_mask)
#    n_v_b_m=tf.bitwise.bitwise_and(values_to_change,not_mask)
#    print('n_v_b_m',n_v_b_m)
#    tensor=tf.bitwise.bitwise_and(n_v_b_m,val_afec_0_Error_to_change)
#    print('n_v_b_1', tensor)
#    tensor_round=tf.bitwise.bitwise_or(tensor,val_afec_1_Error_to_change)
#    #tensor_round  = tf.where(tf.greater_equal(tensor_round,shift), shift-tensor_round , tensor_round )
#    return tensor_round

In [9]:
# Conversión del tensor generado por la red a naturales: para ello se multiplica el número generado por la red que es un 
#un número real x 2**bit de la parte fraccionaria colocando el valor en un entero de 32 bits(de esta forma se convierten a 
# enteros) luego para llevarlo a naturales(a los valores negativos se les suma 2**al tamaño de la palabra que en este caso
# es de 16 bits)

In [2]:
tensor_original= tf.convert_to_tensor([ 1.85546875e-02,  1.85546875e-02, 1.85546875e-02,  1.85546875e-02, -4.88281250e-04, -4.88281250e-04,
 -4.88281250e-04, -4.88281250e-04 ,-4.88281250e-04, -4.88281250e-04, -4.88281250e-04, -4.88281250e-04, -4.88281250e-04, -4.88281250e-04,
 -4.88281250e-04, -4.88281250e-04, -4.88281250e-04, -4.88281250e-04,  3.36914062e-02, -3.51562500e-02 , 1.31835938e-02 , 1.31835938e-02,
  3.22265625e-02 , 3.22265625e-02 , 3.22265625e-02 , 1.31835938e-02, -1.06445312e-01, -1.06445312e-01] )


word_size=16
frac_size=11
 
Ogdtype = tensor_original.dtype
shift   = 2**(word_size-1)
factor  = 2**frac_size


tensor_original  = tf.cast(tensor_original*factor,dtype=tf.int32)
print('tensor despues del cast',tensor_original)
tensor_original  = tf.where(tf.less(tensor_original, 0), -tensor_original + shift , tensor_original )
print('tensor_original',tensor_original)
tensor_actu=tensor_original
print(tf.size(tensor_original))
values_to_run=tf.Variable(0, dtype=tensor_original.dtype)
print(values_to_run)

static_0_Error=tf.convert_to_tensor([49087 ,49087 ,49087 ,49087,65471,49151,65535,65535,65535,65535,49151,65471,65535,65471,65535,65535,65535,49151,49087,32639,32767,65017,65471 ,61439,65535,61423,65527,49151])
static_1_Error=tf.convert_to_tensor([0,0,16384, 64,514,16448,16448,49087,0,16448, 64, 16384, 16448,16384,16448,16448,16448, 64, 0, 0, 128,1024,16384, 16, 4112, 0,2048, 64])
print(tf.size(static_1_Error))
print(tf.size(static_0_Error))
print('static_1_Error',static_1_Error)
print('static_0_Error',static_0_Error)

## Se realiza el proceso de inyeccción de los errores al tensor origunal realisando las operaciones btw correspondientes
tensor_with_error = tf.bitwise.bitwise_and(tensor_original, static_0_Error)
tensor_with_error = tf.bitwise.bitwise_or(tensor_original, static_1_Error)
print( 'tensor_with_error',tensor_with_error)

tensor despues del cast tf.Tensor(
[  38   38   38   38   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
   -1   -1   -1   -1   69  -72   27   27   66   66   66   27 -218 -218], shape=(28,), dtype=int32)
tensor_original tf.Tensor(
[   38    38    38    38 32769 32769 32769 32769 32769 32769 32769 32769
 32769 32769 32769 32769 32769 32769    69 32840    27    27    66    66
    66    27 32986 32986], shape=(28,), dtype=int32)
tf.Tensor(28, shape=(), dtype=int32)
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
tf.Tensor(28, shape=(), dtype=int32)
tf.Tensor(28, shape=(), dtype=int32)
static_1_Error tf.Tensor(
[    0     0 16384    64   514 16448 16448 49087     0 16448    64 16384
 16448 16384 16448 16448 16448    64     0     0   128  1024 16384    16
  4112     0  2048    64], shape=(28,), dtype=int32)
static_0_Error tf.Tensor(
[49087 49087 49087 49087 65471 49151 65535 65535 65535 65535 49151 65471
 65535 65471 65535 65535 65535 49151 49087 32639 32767 65017 65471 61439
 655

In [ ]:
t_or_  38    38    38    38 32769 32769 32769 32769 32769 32769 32769 32769  32769 32769 32769 32769 32769 32769    69 32840    27    27    66    66     66    27 32986 32986

In [ ]:
t_error 38    38 16422   102 33283 49217 49217 49087 32769 49217 32833 49153  49217 49153 49217 49217 49217 32833    69 32840   155  1051 16450    82   4178    27 35034 32986]

In [ ]:
 r_act_ 38    38 16384    64 33282 49216 49216 49087 32769 49216 32832 49152  49216 49152 49216 49216 49216 32832    69 32840   128  1024 16384    80    4112    27 34816 32986], shape=(28,), dtype=int32)

In [11]:
int('1000000000000000',2)

32768

##Prueba para entrar a errores por 0

In [6]:

tensor_original=tf.convert_to_tensor([40167,61311,64511,22,17,37,30,38 ])
print('tensor_original',tensor_original)
static_0_Error=tf.convert_to_tensor([7927,28671,57343,65535,65518,65535,49087 ,49087])
static_1_Error=tf.convert_to_tensor([528,0,1024,12,4,32768,0,0 ])
print(tf.size(static_1_Error))
print(tf.size(static_0_Error))
print('static_1_Error',static_1_Error)
print('static_0_Error',static_0_Error)

## Se realiza el proceso de inyeccción de los errores al tensor origunal realisando las operaciones btw correspondientes
tensor_with_error = tf.bitwise.bitwise_and(tensor_original, static_0_Error)
print('tensor_with_error_and',tensor_with_error)
tensor_with_error = tf.bitwise.bitwise_or(tensor_with_error, static_1_Error)
print( 'tensor_with_error_0r',tensor_with_error)

tensor_original tf.Tensor([40167 61311 64511    22    17    37    30    38], shape=(8,), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
static_1_Error tf.Tensor([  528     0  1024    12     4 32768     0     0], shape=(8,), dtype=int32)
static_0_Error tf.Tensor([ 7927 28671 57343 65535 65518 65535 49087 49087], shape=(8,), dtype=int32)
tensor_with_error_and tf.Tensor([ 7399 28543 56319    22     0    37    30    38], shape=(8,), dtype=int32)
tensor_with_error_0r tf.Tensor([ 7927 28543 57343    30     4 32805    30    38], shape=(8,), dtype=int32)


In [ ]:
v_b 0000000000100101

In [ ]:
7927 28543 57343    38    38 16422   102 33283 49217 49217 49087 32769  49217 32833 49153 49217 49153 49217 49217 49217 32833     5    72   155
  1049 16386    82  4178    11 35026 32986

In [ ]:
24 15 37

In [ ]:
 7927 28671 57343    38    38 32703 16383 65471 65535 65535 65535 32769  65535 49151 65471 65535 65471 65535 65535 65535 49151 63 32639 32767
 32249 32703 28671 32767    15 65527 32986]

In [10]:
bin(65536    )

'0b10000000000000000'

In [ ]:
1000000000000000

In [9]:
65536

65536

In [5]:
int('1000000000000000',2)

32768

## Declaración de variables y preparación de los datos que entran al ciclo

In [ ]:
values_to_run
mask
v_m
v_b
mask_0
valores_afectados
matriz_control_cambios
valores_afec_static_0_Error
valores_afec_static_1_Error
vb_and_mask
vm_and_mask
tensor_act
values_to_run

In [7]:
#Se crea un tensor variable donde se guardarán los valores en cada ciclo que aún no se han encontrado la posición con error
#y hay que continuar analizando con mask>>1
# se cra un tensor con el valos de la máscara inicial la cual es 1000000000000000
values_to_run=tf.Variable(0)
mask=tf.convert_to_tensor(32768)

#crea un tensor de (T and F),donde en T están los valores afectados por los errores inyectados
# esta matriz es la que controla si ya no hay cambios pendientes porque todos los valores han sido analizados y
#detiene el ciclo para no iterar innecesariamente

valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)
print('valores_afectados',valores_afectados)  

# Se extraen los valores que fueron afectados por los errores
v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True))
print('valores afectados por los errores' ,v_m)

#valores que se deberian escribir
v_b=tf.gather_nd(tensor_original,tf.where(valores_afectados==True))
print('valores que se deberian escribir' ,v_b)
#a=np.size(v_m)
# Creo una máscara de 0 del con la cantidad de errores afectados para ir llevando el contrl de los cambios 
#mask_0=tf.convert_to_tensor([0]*a)
mask_0=tf.constant(0)
print('mask_0',mask_0)

# inicializo los valores de comienzo y fin del ciclo teniendo en cuenta que pueden haber errores hasta la posición 15 de
# una palabra
i=tf.constant(0)
out=tf.constant(16)
print(' out inicial',(out))
print('i inicial', i)


#matriz_control(esta la cree pero al final no la uso luego que verifique todo puede ser eliminada sino hace falta)
#matriz_control=tf.equal(tensor_original,tensor_with_error)
matriz_control_cambios=tf.equal(v_b,v_m)
print('matriz_control_cambios',matriz_control_cambios)

# Tensores con los valores de errores en 0 y 1 que propiciaron cambios en el tensor original
valores_afec_static_0_Error=tf.gather_nd(static_0_Error, tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
valores_afec_static_1_Error=tf.gather_nd(static_1_Error, tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
print('valores_afec_static_0_Error',valores_afec_static_0_Error)
print('valores_afec_static_1_Error',valores_afec_static_1_Error)

vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
print('vb_and_mask',vb_and_mask)
vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
vm_and_mask
##### 'aquí actualizo el tensor con los nuevos valores de igual a mask'
print('vm_and_mask',vm_and_mask)
#tf.gather_nd(v_b,tf.where(tf.equal(vb_and_mask,vb_and_mask)):
#a=np.size(v_m)

## la primera vez tensor_actu= tensor original creo esta variable para ir actualizando el tensor con los nuevos valores 
#generados con la técnica del redeondeo

tensor_act=tensor_original 

valores_afectados tf.Tensor([ True  True  True  True  True  True False False], shape=(8,), dtype=bool)
valores afectados por los errores tf.Tensor([ 7927 28543 57343    30     4 32805], shape=(6,), dtype=int32)
valores que se deberian escribir tf.Tensor([40167 61311 64511    22    17    37], shape=(6,), dtype=int32)
mask_0 tf.Tensor(0, shape=(), dtype=int32)
 out inicial tf.Tensor(16, shape=(), dtype=int32)
i inicial tf.Tensor(0, shape=(), dtype=int32)
matriz_control_cambios tf.Tensor([False False False False False False], shape=(6,), dtype=bool)
valores_afec_static_0_Error tf.Tensor([ 7927 28671 57343 65535 65518 65535], shape=(6,), dtype=int32)
valores_afec_static_1_Error tf.Tensor([  528     0  1024    12     4 32768], shape=(6,), dtype=int32)
vb_and_mask tf.Tensor([32768 32768 32768     0     0     0], shape=(6,), dtype=int32)
vm_and_mask tf.Tensor([    0     0 32768     0     0 32768], shape=(6,), dtype=int32)


In [14]:
## depurar para colocar en o donde hay problema en el primer bit y luego que entre al ciclo esto va a ser una función 
a
b
c
values_to_change
val_afec_0_Error_to_change
val_afec_1_Error_to_change
index
val
error_a_0
index_values_error_a_0
tensor_round
val_0

In [ ]:
## casos para el signo tanto para 0 en la primera posición como q

In [8]:
a=tf.math.not_equal(vb_and_mask,vm_and_mask)
print('estoy calculando a y definiendo xq esta dado el error')
print('a vb_and_mask!=vm_and_mask',a)
print('mask',mask)
#values_to_run=tf.gather_nd(v_b,tf.where(a==False))
#values_to_run_with_error=tf.gather_nd(v_m,tf.where(a==False))
#valores_afec_static_0_Error_to_run=tf.gather_nd(valores_afec_static_0_Error,tf.where(a==False))
#print('valores_afec_static_0_Error_to_run',valores_afec_static_0_Error_to_run)
#valores_afec_static_1_Error_to_run=tf.gather_nd(valores_afec_static_1_Error,tf.where(a==False))
#print('valores_afec_static_1_Error_to_run',valores_afec_static_1_Error_to_run)
b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
print('b Vm>Vb',b)
c=tf.where(tf.logical_and(a,b)==True)
#c=tf.where(tf.equal(a,b))
#c=tf.where(tf.equal(tf.where(a==True),tf.where(b==True)))
##++ Hasta aqui es una función que ppuede ser llamada pues se repite 
print('c',c)
print('tamaño de c',tf.size(c))
print(' c Indices de los valores a transformar a 0',c)
#mask_0=tf.tensor_scatter_nd_update(mask_0,c,tf.convert_to_tensor([1]*np.size(c)))

#print('mask_0 desdes update', mask_0)
#out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
values_to_change=tf.gather_nd(v_b,c)
mask_0=tf.add(mask_0,tf.size(values_to_change))
print('mask_0 desdes update', mask_0)
val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
print('values_to_change',values_to_change)
index=c.numpy()
print('cnumpy',index)
print('valores_afectados',valores_afectados)
val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
print('val',val)



#### agraegado para si se encuentra error a 0 en la posición más significativa  esto 
error_a_0=tf.math.greater(v_b,v_m)
print('error_a_0',error_a_0)
index_values_error_a_0=tf.where(tf.logical_and(a,error_a_0)==True)
#index_values_error_a_0=tf.where(tf.equal(a,error_a_0))
#index_values_error_a_0=tf.where(error_a_0==True)
print('index_values_error_a_0',index_values_error_a_0)
### esto será una función
values_to_change=tf.gather_nd(v_b,index_values_error_a_0)
val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,index_values_error_a_0)
print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,index_values_error_a_0)
print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
print('values_to_change',values_to_change)
#tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
#print('tensor_round',tensor_round)
index_0=index_values_error_a_0.numpy()
print('cnumpy',index_0)
print('valores_afectados',valores_afectados)
val_0=tf.gather_nd(tf.where(valores_afectados),index_0)## tomo los indices del tensor original qu ese cambiaran
print('val_0',val_0)
if np.size(val_0)>0:
    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val_0, tf.convert_to_tensor([0]*np.size(val_0) ))
    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val_0,tf.convert_to_tensor([False]*np.size(val_0)))
    print('valores_afectados dentro de error 0 en primera posición',valores_afectados )
    mask_0=tf.add(mask_0,tf.size(values_to_change))
    print('tensor_act funciion a 0',tensor_act)
    print('mask_0',mask_0)
    print('valores_afectados',valores_afectados)
    print('tensor_with_error',tensor_with_error)
    #tensor_with_error_run= tf.gather_nd(tensor_with_error,tf.where(valores_afectados ==True))   
    v_b=tf.gather_nd(tensor_act,tf.where(valores_afectados==True))
    print('v_b',v_b)
    v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True) )
    print('v_m',v_m)
    valores_afec_static_0_Error=tf.gather_nd(static_0_Error,tf.where(valores_afectados==True))
    print('valores_afec_static_0_Error', valores_afec_static_0_Error)
    valores_afec_static_1_Error=tf.gather_nd(static_1_Error,tf.where(valores_afectados==True))
    print('valores_afec_static_1_Error', valores_afec_static_0_Error)
    #tensor_with_error=tensor_with_error_run
if np.size(val)> 0: 
    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tf.convert_to_tensor([0]*np.size(val) ) ) 
    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
    mask_0=tf.add(mask_0,tf.size(values_to_change))
    #tensor_with_error_run= tf.gather_nd(tensor_with_error,tf.where(valores_afectados ==True))   
    v_b=tf.gather_nd(tensor_act,tf.where(valores_afectados==True))
    print('v_b',v_b)
    v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True) )
    print('v_m',v_m)
    valores_afec_static_0_Error=tf.gather_nd(static_0_Error,tf.where(valores_afectados==True))
    print('valores_afec_static_0_Error', valores_afec_static_0_Error)
    valores_afec_static_1_Error=tf.gather_nd(static_1_Error,tf.where(valores_afectados==True))
    print('valores_afec_static_1_Error', valores_afec_static_0_Error)
    
print('tensor_act',tensor_act)
print('valores_afectados',valores_afectados)
print('tensor_with_error',tensor_with_error)
print('v_b',v_b)
print('v_m',v_m)
print('valores_afec_static_0_Error', valores_afec_static_0_Error)
print('valores_afec_static_1_Error',valores_afec_static_1_Error)
mask=tf.bitwise.right_shift(mask,1)

estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([ True  True False False False  True], shape=(6,), dtype=bool)
mask tf.Tensor(32768, shape=(), dtype=int32)
b Vm>Vb tf.Tensor([False False False  True False  True], shape=(6,), dtype=bool)
c tf.Tensor([[5]], shape=(1, 1), dtype=int64)
tamaño de c tf.Tensor(1, shape=(), dtype=int32)
 c Indices de los valores a transformar a 0 tf.Tensor([[5]], shape=(1, 1), dtype=int64)
mask_0 desdes update tf.Tensor(1, shape=(), dtype=int32)
val_afec_0_Error_to_change tf.Tensor([65535], shape=(1,), dtype=int32)
val_afec_1_Error_to_change tf.Tensor([32768], shape=(1,), dtype=int32)
values_to_change tf.Tensor([37], shape=(1,), dtype=int32)
cnumpy [[5]]
valores_afectados tf.Tensor([ True  True  True  True  True  True False False], shape=(8,), dtype=bool)
val tf.Tensor([[5]], shape=(1, 1), dtype=int64)
error_a_0 tf.Tensor([ True  True  True False  True False], shape=(6,), dtype=bool)
index_values_error_a_0 tf.Tensor(


In [ ]:
vb 40167,61311,64511
error_0: 7927,28671,57343
error_1: 528,0,1024
0.Antes de entrar al ciclo
1001110011100111 vb 40167
0001111011110111 vm 7927
0001111011110111 error_0 7927
0001111011110111 and
0000001000010000 error_1 528
0001111011110111 7927
Prueba
mask=32768     
n_mask=mask-1   0111111111111111
1001110011100111 vb
0111111111111111   n_mask
1111111111111111 or 65535
0001111011110111error_0 7927
0001111011110111 and  7927
0000001000010000 or error_1
0001111011110111 7927


1. Antes de entrar al ciclo
1110111101111111 vb 61311
0110111111111111 error_0 28671
0110111101111111 28543
0000000000000000 error_1 
0110111101111111 28543
Prueba
mask=32768     
n_mask=(mask-1) 0111111111111111
1110111101111111 vb
0111111111111111 n_mask
1111111111111111 or 65535
0110111111111111 and error_0 28671
0110111111111111
0000000000000000 error_1
0110111111111111 or  28671 

3. iteración i=1
1111101111111111 vb 64511
1101111111111111 v_m 57343
1101111111111111 error_0 57343
1101101111111111
0000010000000000 error_1 1024
1101111111111111 57343

Prueba
mask=8192 0010000000000000    
n_mask=(mask-1)0001111111111111
1111101111111111 vb 64511
0001111111111111  n_mask
1111111111111111 or n_mask
1101111111111111 error_0 57343
1101111111111111 and
0000010000000000 error_1 1024
1101111111111111  57343




1111101111111111v_b
1000000000000000
1000000000000000 vb_mask
1101111111111111- v_m
1000000000000000
1000000000000000 vm_mask
vb_mask!=vm_mask  false
v_m>v_b false



In [34]:
bin( 8191)

'0b1111111111111'

In [35]:
int('1101111111111111',2)

57343

## modificarlo porque en el caso donde hay 0 en la posición 1 no lo estaba contemplando

In [ ]:
values_to_run_with_error
valores_afec_static_0_Error_to_run
valores_afec_static_0_Error_to_run
values_to_change

In [9]:
def while_body(i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error):
    print('dentro del body')
    print('i',i)
    print('out que llega al body', out)
    print('tensor_act',tensor_act)
    print('tensor_original',tensor_original)
    print('mask',mask)
    print('mask_0',mask_0)
    print('v_m',v_m)
    print('v_b',v_b)
    print('tensor_with_error',tensor_with_error)
    #hay_cambios=tf.math.not_equal(tensor_original,tensor_act)
    #print('hay_cambios',hay_cambios)
    #stop_condition=tf.experimental.numpy.any(valores_afectados)
    #if stop_condition==False:
        
    ## Este if lo cree por si hay cambios antes de llegar al ciclo por haber errores e la posición 15 pero quizás ya o sea
    # necesraio analizar luego de experimentar  si mask_0 llega con valores a 1 es porque se hicieron cambios y se deben
    #analizar los btw con los valores resttantes sino todo se mantiene con los valores originales de entrada.
    if tf.reduce_sum(mask_0)>0:
        print('entre en condicion mask_0>0')
        print('valores donde debo correr el mask')
        
        vb_and_mask=tf.bitwise.bitwise_and(v_b,mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(v_m,mask)
        print('vm_and_mask',vm_and_mask)  
    else:
        
         ### esta estructura cambiaria porque ya tendre solo los valores con problemas, tomaria valores de tensores variables 
         #inicializados en 0 hasta que tome sus valores luego del recorrido por el body
         ## el tensor control tomara un papel aca porque luego actualizaria el tensor_actu en los posiciones donde 
             #tensorcontrol sea false mas otra condicion mas o debo ir comparando el tensor actu con el original y envio los dos
        #valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)## inicialmente no hara nada xq son iguales
        print('entre en el else')
        vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
        print('vm_and_mask',vm_and_mask) 
        print('v_b',v_b)
        print('v_m',v_m)
        
    a=tf.math.not_equal(vb_and_mask,vm_and_mask)
    print('estoy calculando a y definiendo xq esta dado el error')
    print('a vb_and_mask!=vm_and_mask',a)
    values_to_run=tf.gather_nd(v_b,tf.where(a==False))
    values_to_run_with_error=tf.gather_nd(v_m,tf.where(a==False))
    valores_afec_static_0_Error_to_run=tf.gather_nd(valores_afec_static_0_Error,tf.where(a==False))
    print('valores_afec_static_0_Error_to_run',valores_afec_static_0_Error_to_run)
    valores_afec_static_1_Error_to_run=tf.gather_nd(valores_afec_static_1_Error,tf.where(a==False))
    print('valores_afec_static_1_Error_to_run',valores_afec_static_1_Error_to_run)
    print('values_to_run', values_to_run)
    print(' values_to_run_with_error', values_to_run_with_error)
    #valores_afectados_to_run= valores_afectados
    #u=tf.size(values_to_run)>0
    
    
    b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
    print('b Vm>Vb',b)
    c=tf.where(tf.logical_and(a,b)==True)
    #c=tf.where(tf.equal(a,b))
    print('tamaño de c',tf.size(c))
    print(' c Indices de los valores a transformar con la variante error a 1',c)
    
    error_a_0=tf.math.greater(v_b,v_m)
    print('error_a_0',error_a_0)
    index_values_error_a_0=tf.where(tf.logical_and(a,error_a_0)==True)
    #index_values_error_a_0=tf.where(error_a_0==True)
    print('index_values_error_a_0',index_values_error_a_0)
    print('vb',v_b)
    
        
   
    if tf.size(c)>0:
        
        values_to_change=tf.gather_nd(v_b,c)
        val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
        print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
        print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        print('values_to_change',values_to_change)
        tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        print('tensor_round',tensor_round)
        index=c.numpy()
        print('indice a camniar tensor act',index)
        print('valores_afectados',valores_afectados)
        val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
        print('val',val)
        tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
        valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
        mask_0=tf.add(mask_0,tf.size(values_to_change))
        print('mask_0 desdes update', mask_0)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)                                              
    
    
        
        
        
    
        
    ##no es el que esta en mask en esse momento, retorno new mask, new_mask_0, tensor_act
    if tf.size(index_values_error_a_0)>0: 
        print('dentro detf.size(error_a_0)>0 ')
        #out=tf.constant(tf.math.subtract(out,tf.size(error_a_0)))
        #print('out',out)
        #mask_0=tf.tensor_scatter_nd_update(mask_0,error_a_0,tf.convert_to_tensor([1]*np.size(error_a_0)))
        print()
        #out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
        values_to_change=tf.gather_nd(v_b,index_values_error_a_0)
        val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,index_values_error_a_0)
        print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,index_values_error_a_0)
        print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        print('values_to_change',values_to_change)
        tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        index_0=index_values_error_a_0.numpy()
        print('index_0',index_0)
        print('valores_afectados',valores_afectados)
        val_0=tf.gather_nd(tf.where(valores_afectados),index_0)## tomo los indices del tensor original qu ese cambiaran
        print('val_0',val_0)
        
        tensor_act = tf.tensor_scatter_nd_update(tensor_act, val_0, tensor_round )
        valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val_0,tf.convert_to_tensor([False]*np.size(val_0)))
        #print('tensor_round en el body',tensor_round)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)        
        mask_0=tf.add(mask_0,tf.size(val_0))
        print('mask_0 desdes update', mask_0)
    
        
        
        
    if tf.size(values_to_run)>0:
        #mask=mask>>1
        mask=tf.bitwise.right_shift(mask,1)
        print('dentro del if values to run',values_to_run)
        print('mask',mask)
        v_b=values_to_run
        v_m=values_to_run_with_error 
        valores_afec_static_0_Error=valores_afec_static_0_Error_to_run
        valores_afec_static_1_Error=valores_afec_static_1_Error_to_run
        print('v_b',v_b)
        print('v_m',v_m)
        print('valores_afec_static_0_Error',valores_afec_static_0_Error)
        print('valores_afec_static_1_Error',valores_afec_static_1_Error)
        
    
    
    if tf.experimental.numpy.any(valores_afectados)==False:
        
        print('tensor_act',tensor_act)
        i=16
        
    print(i) 
    
    
    return (i+1),out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error ## retorno los valores que aun no se han modificado por ninguna d elas variantes puesto que el bit con error     
    
i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error= tf.while_loop(
    lambda i, *_ : tf.less(i,out),  # condición -> revisar cada bit
     while_body,  # aqui todo lo que tengas que hacer de comparar los vectores (que estos los puedes definir por fuera
                 # o pasarlos a la misma función de while_body y no modificarlos)
    (i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error)  # valores iniciales
)   



dentro del body
i tf.Tensor(0, shape=(), dtype=int32)
out que llega al body tf.Tensor(16, shape=(), dtype=int32)
tensor_act tf.Tensor([    0     0 64511    22    17     0    30    38], shape=(8,), dtype=int32)
tensor_original tf.Tensor([40167 61311 64511    22    17    37    30    38], shape=(8,), dtype=int32)
mask tf.Tensor(16384, shape=(), dtype=int32)
mask_0 tf.Tensor(5, shape=(), dtype=int32)
v_m tf.Tensor([57343    30     4], shape=(3,), dtype=int32)
v_b tf.Tensor([64511    22    17], shape=(3,), dtype=int32)
tensor_with_error tf.Tensor([ 7927 28543 57343    30     4 32805    30    38], shape=(8,), dtype=int32)
entre en condicion mask_0>0
valores donde debo correr el mask
vb_and_mask tf.Tensor([16384     0     0], shape=(3,), dtype=int32)
vm_and_mask tf.Tensor([16384     0     0], shape=(3,), dtype=int32)
estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([False False False], shape=(3,), dtype=bool)
valores_afec_static_0_Error_to_run tf.Tens

## Ciclo este corre ok sin tener en cuenta lo de los errores en 0 en la primera posición

In [6]:
def while_body(i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error):
    print('dentro del body')
    print('i',i)
    print('out que llega al body', out)
    print('tensor_act',tensor_act)
    print('tensor_original',tensor_original)
    print('mask',mask)
    print('mask_0',mask_0)
    #hay_cambios=tf.math.not_equal(tensor_original,tensor_act)
    #print('hay_cambios',hay_cambios)
    #stop_condition=tf.experimental.numpy.any(valores_afectados)
    #if stop_condition==False:
        
    ## Este if lo cree por si hay cambios antes de llegar al ciclo por haber errores e la posición 15 pero quizás ya o sea
    # necesraio analizar luego de experimentar  si mask_0 llega con valores a 1 es porque se hicieron cambios y se deben
    #analizar los btw con los valores resttantes sino todo se mantiene con los valores originales de entrada.
    if tf.reduce_sum(mask_0)>0:
        print('entre en condicion mask_0>0')
        print('valores donde debo correr el mask')
        
        vb_and_mask=tf.bitwise.bitwise_and(v_b,mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(v_m,mask)
        print('vm_and_mask',vm_and_mask)  
    else:
        
         ### esta estructura cambiaria porque ya tendre solo los valores con problemas, tomaria valores de tensores variables 
         #inicializados en 0 hasta que tome sus valores luego del recorrido por el body
         ## el tensor control tomara un papel aca porque luego actualizaria el tensor_actu en los posiciones donde 
             #tensorcontrol sea false mas otra condicion mas o debo ir comparando el tensor actu con el original y envio los dos
        valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)## inicialmente no hara nada xq son iguales
        print('entre en el else')
        vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
        print('vm_and_mask',vm_and_mask) 
        print('v_b',v_b)
        print('v_m',v_m)
        
    a=tf.math.not_equal(vb_and_mask,vm_and_mask)
    print('estoy calculando a y definiendo xq esta dado el error')
    print('a vb_and_mask!=vm_and_mask',a)
    values_to_run=tf.gather_nd(v_b,tf.where(a==False))
    values_to_run_with_error=tf.gather_nd(v_m,tf.where(a==False))
    valores_afec_static_0_Error_to_run=tf.gather_nd(valores_afec_static_0_Error,tf.where(a==False))
    print('valores_afec_static_0_Error_to_run',valores_afec_static_0_Error_to_run)
    valores_afec_static_1_Error_to_run=tf.gather_nd(valores_afec_static_1_Error,tf.where(a==False))
    print('valores_afec_static_1_Error_to_run',valores_afec_static_1_Error_to_run)
    
    print(' values_to_run_with_error', values_to_run_with_error)
    #valores_afectados_to_run= valores_afectados
    u=tf.size(values_to_run)>0
    
    
    b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
    print('b Vm>Vb',b)
    c=tf.where(tf.equal(a,b))
    print('tamaño de c',tf.size(c))
    print(' c Indices de los valores a transformar con la variante error a 1',c)
    
    error_a_0=tf.math.greater(v_b,v_m)
    print('error_a_0',error_a_0)
    index_values_error_a_0=tf.where(error_a_0==True)
    print('index_values_error_a_0',index_values_error_a_0)
    
        
   
    if tf.size(c)>0:
        #out=tf.constant(tf.math.subtract(out,tf.size(c)))
        #print('out',out)
        #mask_0=tf.tensor_scatter_nd_update(mask_0,c,tf.convert_to_tensor([1]*np.size(c)))
        
        #out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
        values_to_change=tf.gather_nd(v_b,c)
        val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
        print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
        print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        print('values_to_change',values_to_change)
        tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        print('tensor_round',tensor_round)
        index=c.numpy()
        print('cnumpy',index)
        print('valores_afectados',valores_afectados)
        val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
        print('val',val)
        tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
        valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
        mask_0=tf.add(mask_0,tf.size(values_to_change))
        print('mask_0 desdes update', mask_0)
        
        #print('tensor_round en el body',tensor_round)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)                                              
    
    
        #mask=tf.bitwise.right_shift(mask,1)
        #print('mask despues del shift',mask)
    
        print('tf.less dentro del body',a)
        #tensor_act = ...## llamo a una funciíon a la que le paso los indices en el tensor original que cambie y los nuevos valores 
        #y hago el update
        #new_mask = mask>>1
        #print(new_mask)
        #mask = new_mask_binary 
        #i = sum_new_mask_binary
        print('out',out)
        print('i',i)
        
    
        
    ##no es el que esta en mask en esse momento, retorno new mask, new_mask_0, tensor_act
    if tf.size(index_values_error_a_0)>0: 
        print('dentro detf.size(error_a_0)>0 ')
        #out=tf.constant(tf.math.subtract(out,tf.size(error_a_0)))
        #print('out',out)
        #mask_0=tf.tensor_scatter_nd_update(mask_0,error_a_0,tf.convert_to_tensor([1]*np.size(error_a_0)))
        
        #out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
        values_to_change=tf.gather_nd(v_b,error_a_0)
        val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,error_a_0)
        print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,error_a_0)
        print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        print('values_to_change',values_to_change)
        tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        index=error_a_0.numpy()
        print('cnumpy',index)
        print('valores_afectados',valores_afectados)
        val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
        print('val',val)
        
        tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
        valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
        #print('tensor_round en el body',tensor_round)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)        
        mask_0=tf.add(mask_0,tf.size(values_to_change))
        print('mask_0 desdes update', mask_0)
    
        #mask=tf.bitwise.right_shift(mask,1)
        #print('mask despues del shift',mask)
    
        
        #tensor_act = ...## llamo a una funciíon a la que le paso los indices en el tensor original que cambie y los nuevos valores 
        #y hago el update
        #new_mask = mask>>1
        #print(new_mask)
        #mask = new_mask_binary 
        #i = sum_new_mask_binary
        #print('out',out)
       # print('i',i)
    if tf.size(values_to_run)>0:
        #mask=mask>>1
        mask=tf.bitwise.right_shift(mask,1)
        print('dentro del if values to run',values_to_run)
        print('mask',mask)
        v_b=values_to_run
        v_m=values_to_run_with_error 
        valores_afec_static_0_Error=valores_afec_static_0_Error_to_run
        valores_afec_static_1_Error=valores_afec_static_1_Error_to_run
    #i=tf.add(i, 1) 
    #print(i)
    
    if tf.experimental.numpy.any(valores_afectados)==False:
        #factor=2**11
        #tensor_act= tf.cast(tensor_act/factor,dtype=tf.float32)
        print(tensor_act)
        i=16
        
    print(i) 
    
    
    return (i+1),out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error ## retorno los valores que aun no se han modificado por ninguna d elas variantes puesto que el bit con error     
    
i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error= tf.while_loop(
    lambda i, *_ : tf.less(i,out),  # condición -> revisar cada bit
     while_body,  # aqui todo lo que tengas que hacer de comparar los vectores (que estos los puedes definir por fuera
                 # o pasarlos a la misma función de while_body y no modificarlos)
    (i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error)  # valores iniciales
)   

#Analizar porque donde vm_and mask es igual a mask es porque el error está en la primera posiscíon
#entonces actualizo el tensor con el valor de mask `10000000000000'
#y donde vale 0 hay que seguir corriendo el mask
#hacer esto en otra página y hacerlo con el tensor mas grande para avarcar mas variantes


dentro del body
i tf.Tensor(0, shape=(), dtype=int32)
out que llega al body tf.Tensor(16, shape=(), dtype=int32)
tensor_act tf.Tensor(
[  528     0 64511    38    38 16384    64 33282 49216 49216 49087 32769
 49216 32832 49152 49216 49152 49216 49216 49216 32832    69     0   128
  1024 16384    16  4112    27 34816 32986], shape=(31,), dtype=int32)
tensor_original tf.Tensor(
[40167 61311 64511    38    38    38    38 32769 32769 32769 32769 32769
 32769 32769 32769 32769 32769 32769 32769 32769 32769    69 32840    27
    27    66    66    66    27 32986 32986], shape=(31,), dtype=int32)
mask tf.Tensor(16384, shape=(), dtype=int32)
mask_0 tf.Tensor(24, shape=(), dtype=int32)
entre en condicion mask_0>0
valores donde debo correr el mask
vb_and_mask tf.Tensor(
[    0 16384 16384     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0], shape=(27,), dtype=int32)
vm_and_mask tf.Tensor(
[    0 16384 1

InvalidArgumentError: indices[16] = [25] does not index into param shape [3,1], node name: GatherNd [Op:GatherNd]

In [35]:
bin(32767)

'0b111111111111111'

In [36]:
int('0010000000000000',2)

8192

In [ ]:
38     38  16384     64   -514 -16448 -16448 -16319  32769 -16448     -64 -16384 -16448 -16384 -16448 -16448 -16448    -64     69  32840
    128   1024  16384     80   4112     27  -2048  32986

In [ ]:
te_or= 38    38    38    38 32769 32769 32769 32769 32769 32769 32769 32769   32769 32769 32769 32769 32769 32769    69 32840    27    27    66    66      66    27 32986 32986

te_err= 49087 49087 49087 49087 65471 49151 65535 65535 65535 65535 49151 65471  65535 65471 65535 65535 65535 49151 49151 65407 32767 65019 65535 61439  65535 61439 65535 49151

In [ ]:
va_afec_0_Error =0     0   16384    64   514   16448  16448  49087     0  16448    64  16384  16448 16384 16448 16448 16448    64     0     0   128  1024 16384    16   4112     0  2048    64], 
val_afec_1_Error=49087 49087 49087   49087 65471  49151  65535 65535 65535  65535   49151 65471  65535 65471 65535 65535 65535 49151 49087 32639 32767 65017 65471 61439  65535 61423 65527 49151], 

In [ ]:
[ 4] [ 6] [ 7] [ 8] [ 9] [11] [12] [13] [14] [15] [16] [19] [20] [26]

In [84]:
bin(49152 )

'0b1100000000000000'

In [92]:
n_mask=16383
not_mask=np.invert(np.array([n_mask], dtype=np.uint16))
not_mask

array([49152], dtype=uint16)

In [ ]:
i=0   [ 0] [ 1] [ 2] [ 3] [18] [21] [22] [23] [24] [25]

In [ ]:
v_b=   0000000000100110  38
v_m=   1011111110111111  49087  cambio en la iteración 0 
mask=  1000000000000000
mask-1=0111111111111111 32767
n_mask=1000000000000000 32768
nvb=v_b and not_mask
0000000000100110 
1000000000000000
1000000000000000 and  32768
0000000000000000 error_0
0000000000000000 btw_and
1011111110111111 error_1 
1011111110111111 btw_or 49087
shift-49087=-16319


In [85]:
bin(16383 )

'0b11111111111111'

In [83]:
bin(49152)

'0b11111111111111'

In [86]:
int('1100000000000000',2)

49152

In [ ]:
i=1 [ 4] [ 6] [ 7] [ 8] [ 9] [11] [12] [13] [14] [15] [16] [19] [20] [26]


In [ ]:
tensor_original=  32769  32769 32769 32769 32769  32769   32769 32769 32769 32769 32769  32840    27  32986       

tensor_with err=  65471  65535 65535 65535 65535 65471    65535 65471 65535 65535 65535  65407 32767  65535 

In [ ]:
va_afec_0_Error =   514     16448  49087     0  16448     16384  16448 16384 16448 16448 16448         0   128    2048   , 
val_fec_1_Error=   65471    65535 65535   65535  65535    65471  65535 65471 65535 65535 65535      32639 32767    65527 ], 

In [ ]:
val_afec_0_Err[    514     16448  49087     0    16448    16384  16448  16384 16448 16448 16448     0   128  2048], shape=(14,)
val_afec_1_Error[  65471   65535   65535   65535 65535   65471  65535  65471 65535 65535 65535   32639 32767 65527], shape=(14,), dtype=int32)
values_to_change [32769 32769 32769 32769  32769 32769  32769 3 2769  32769 32769 32769 32840   27 32986], shape=(14,), dtype=int32)

In [44]:
bin(514 )

'0b1000000010'

In [ ]:
v_b=   1000000000000001 32769
v_m=   1111111110111111 65471 iteraciOn 
mask=  0100000000000000  16384
mask-1 0011111111111111  16383
n_mask=1100000000000000  49152
nvb=v_b and not_mask
1000000000000001
1100000000000000
1000000000000000 and 32768
0000001000000010 error_0 514
0000000000000000 btw_and
1111111110111111 error_1 65471
1111111110111111 65471




In [21]:
shift=2**15
shift


32768

In [23]:
shift-65471

-32703

In [ ]:
i=[ 5], [10], [17], [27]

In [ ]:
tensor_original:   32769  32769  32769    32986             
tensor_with_error:[ 49151  49151  49151  49151       

In [ ]:
val_afec_0_Error_to_change tf.Tensor([16448    64    64    64]
val_afec_1_Error_to_change tf.Tensor([49151 49151 49151 49151] 
values_to_change tf.Tensor([32769 32769 32769 32986] 

In [45]:
bin(16448)

'0b100000001000000'

In [43]:
int('1100000001000000',2)

49216

In [ ]:
v_b=   1000000000000001 32769
v_m=   1011111111111111 49151 
mask=  0010000000000000 8192
mask-1 0001111111111111 8191
n_mask=1110000000000000 57344
nvb=v_b and not_mask
1000000000000001
1110000000000000
1000000000000000 and 32768
1011111111111111 error_0 
1000000000000000 btw_and
0100000001000000 error_1 
1100000001000000 49216

In [ ]:
static_1_Error:[  0     0 16384    64   514 16448 16448 49087     0 16448    64 16384  16448 16384 16448 16448 16448    64     0     0   128  1024 16384    16
  4112     0  2048    64]
static_0_Error:[49087 49087 49087 49087 65471 49151 65535 65535 65535 65535 49151 65471  65535 65471 65535 65535 65535 49151 49087 32639 32767 65017 65471 61439
 65535 61423 65527 49151]

In [ ]:
tensor_original:[  38    38    38    38   32769  32769  32769  32769  32769  32769   32769  32769  32769 32769  32769  32769 32769  32769   69  32840    27    27     66      66      66      27     32986        32986]


tensowith_error:[  38    38  16422  102  33283  49217  49217  49087  32769  49217   32833  49153  49217  49153  49217  49217 49217  32833   69  32840   155    1051   16450    82     4178    27      35034      32986]

In [ ]:
                 38    38   16384   64  -514  -16448  -16448 -16319  32769 -16448   -64  -16384 -16448 -16384  -16448 -16448 -16448 -64    69 32840    128   1024    16384   80     4112    27  -2048 32986

In [12]:
bin(102)

'0b1100110'

In [ ]:
0100110
1100110